<a href="https://colab.research.google.com/github/Layravbf/Engenharia-de-Dados/blob/main/cotacao_moeda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import random

In [ ]:
url = 'https://economia.awesomeapi.com.br/json/last/USD-BRL'
ret = requests.get(url)

In [ ]:
if ret:
  print(ret.text)
else:
  print('Falhou')

{"USDBRL":{"code":"USD","codein":"BRL","name":"Dólar Americano/Real Brasileiro","high":"5.1649","low":"5.103","varBid":"-0.0305","pctChange":"-0.59","bid":"5.1332","ask":"5.1349","timestamp":"1659974195","create_date":"2022-08-08 12:56:35"}}


In [ ]:
dolar = json.loads(ret.text)['USDBRL']

In [ ]:
dolar['bid']

'5.1332'

In [ ]:
print(f"20 Dólares hoje custam {float(dolar['bid']) *20} reais")

20 Dólares hoje custam 102.66400000000002 reais


In [ ]:
def cotacao(valor, moeda):
  url = f'https://economia.awesomeapi.com.br/json/last/{moeda}'
  ret = requests.get(url)
  dolar = json.loads(ret.text)[moeda.replace('-', '')]
  print(f"{valor} {moeda[:3]} hoje custam {float(dolar['bid']) *valor} {moeda[4:]}")

In [ ]:
cotacao(20, 'JPY-BRL')

20 JPY hoje custam 0.7618 BRL


In [ ]:
try:
  cotacao(20, 'USD-BRL')
except Exception as e:
  print(f'Chave {e} não encontrada')
else:
  print('ok!')

20 USD hoje custam 102.67200000000001 BRL
ok!


In [ ]:
#try exception
def mult_moedas(valor):
  list_Money = [
      'USD-BRL',
      'EUR-BRL',
      'BTC-BRL',
      'RPL-BLR', #moeda inexistente
      'JPY-BRL'
  ]

  for moeda in list_Money:
    try: 
      url = f'https://economia.awesomeapi.com.br/json/last/{moeda}'
      ret = requests.get(url)
      dolar = json.loads(ret.text)[moeda.replace('-', '')]
      print(f"{valor} {moeda[:3]} hoje custam {float(dolar['bid']) *valor} {moeda[4:]}")
    except:
      print(f'Falha na moeda: {moeda}')

mult_moedas(15)

15 USD hoje custam 76.941 BRL
15 EUR hoje custam 78.486 BRL
15 BTC hoje custam 1851.0 BRL
Falha na moeda: RPL-BLR
15 JPY hoje custam 0.5709 BRL


In [ ]:
#decorador

def error_check(func):
  def inner_func(*args, **kargs):
    try:
      func(*args, **kargs)
    except:
      print(f'{func.__name__} falhou ')
  return inner_func

@error_check
def func_moedas(valor, moeda):
  url = f'https://economia.awesomeapi.com.br/json/last/{moeda}'
  ret = requests.get(url)
  dolar = json.loads(ret.text)[moeda.replace('-', '')]
  print(f"{valor} {moeda[:3]} hoje custam {float(dolar['bid']) *valor} {moeda[4:]}")
  
func_moedas(20, 'USD-BRL')
func_moedas(20, 'EUR-BRL')
func_moedas(20, 'BTC-BRL')
func_moedas(20, 'RPL-BLR')
func_moedas(20, 'JPY-BRL')

20 USD hoje custam 102.436 BRL
20 EUR hoje custam 104.474 BRL
20 BTC hoje custam 2468.0 BRL
func_moedas falhou 
20 JPY hoje custam 0.761 BRL


In [ ]:
#simulando erros de requisição
import backoff
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
  rnd = random.random()
  print(f"""
          RND: {rnd}
          args: {args if args else 'sem agrs'}
          kargs: {kargs if kargs else 'sem kargs'}
      """)
  if rnd < .2:
    raise ConnectionAbortedError('Conexão foi finalizada')
  elif rnd < .4: 
    raise ConnectionRefusedError('Conexão foi recusada')
  elif rnd < .6:
    raise TimeoutError('Tempo de espera excedido')
  else:
    return "OK!"

In [ ]:
test_func()

INFO:backoff:Backing off test_func(...) for 0.8s (ConnectionRefusedError: Conexão foi recusada)



          RND: 0.316336712392962
          args: sem agrs
          kargs: sem kargs
      

          RND: 0.8538027336945508
          args: sem agrs
          kargs: sem kargs
      


'OK!'

In [ ]:
test_func(42)

INFO:backoff:Backing off test_func(...) for 0.6s (ConnectionAbortedError: Conexão foi finalizada)



          RND: 0.10963298881362082
          args: (42,)
          kargs: sem kargs
      

          RND: 0.6272686198012044
          args: (42,)
          kargs: sem kargs
      


'OK!'

In [ ]:
test_func(42, 51, nome='Layra')


          RND: 0.8142995953307742
          args: (42, 51)
          kargs: {'nome': 'Layra'}
      


'OK!'

In [ ]:
# Logs
import logging

log = logging.getLogger()
log.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
log.addHandler(ch)

In [ ]:
import backoff
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
  rnd = random.random()
  log.debug(f" RND: {rnd} ")
  log.info(f"args: {args if args else 'sem agrs'}")
  log.info(f"kargs: {kargs if kargs else 'sem kargs'}")
  if rnd < .2:
    log.error('Conexão foi finalizada')
    raise ConnectionAbortedError('Conexão foi finalizada')
  elif rnd < .4: 
    log.error('Conexão foi recusada')
    raise ConnectionRefusedError('Conexão foi recusada')
  elif rnd < .6:
    log.error('Tempo de espera excedido')
    raise TimeoutError('Tempo de espera excedido')
  else:
    return "OK!"

In [ ]:
test_func()

DEBUG:root: RND: 0.299041233524585 
2022-08-08 17:16:52,192 - root - DEBUG -  RND: 0.299041233524585 
INFO:root:args: sem agrs
2022-08-08 17:16:52,197 - root - INFO - args: sem agrs
INFO:root:kargs: sem kargs
2022-08-08 17:16:52,204 - root - INFO - kargs: sem kargs
ERROR:root:Conexão foi recusada
2022-08-08 17:16:52,209 - root - ERROR - Conexão foi recusada
INFO:backoff:Backing off test_func(...) for 0.5s (ConnectionRefusedError: Conexão foi recusada)
2022-08-08 17:16:52,215 - backoff - INFO - Backing off test_func(...) for 0.5s (ConnectionRefusedError: Conexão foi recusada)
DEBUG:root: RND: 0.748871013255932 
2022-08-08 17:16:52,694 - root - DEBUG -  RND: 0.748871013255932 
INFO:root:args: sem agrs
2022-08-08 17:16:52,698 - root - INFO - args: sem agrs
INFO:root:kargs: sem kargs
2022-08-08 17:16:52,702 - root - INFO - kargs: sem kargs


'OK!'